DATA PRE-PROCESSING

In [1]:
import cv2
import numpy as np
import os

data_dir = "C:/Users/sachi/Documents/aisign/venv/dataset"
image_size = (64, 64)
X, y = [], []

for label, sign in enumerate(os.listdir(data_dir)):
    for img_name in os.listdir(os.path.join(data_dir, sign)):
        img_path = os.path.join(data_dir, sign, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, image_size)
        X.append(img)
        y.append(label)

X = np.array(X) / 255.0  # Normalize
y = np.array(y)


HAND LANDMARKS

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

# Define dataset path (adjust if needed)
data_dir = os.path.abspath("C:/Users/sachi/Documents/aisign/venv/dataset")  # Adjust this if needed
class_folders = os.listdir(data_dir)

# Select the first class folder (e.g., "A") and first image
first_class = class_folders[0]  # Pick first label folder
first_image_path = os.path.join(data_dir, first_class, os.listdir(os.path.join(data_dir, first_class))[0])

# Load the image
image = cv2.imread(first_image_path)  # Read the image
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB for correct colors

# Display the image
plt.imshow(image)
plt.axis("off")
plt.title(f"Sample Image - {first_class}")
plt.show()
